In [1]:
from torchvision import transforms
from PIL import Image
from typing import Any

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib import TaskType
from anomalib.data import MVTec
from anomalib.data.utils import read_image
from anomalib.deploy import ExportType, OpenVINOInferencer
from anomalib.engine import Engine
from anomalib.models import Padim
import os
from pathlib import Path

In [2]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib'

In [3]:
cd anomalib/

d:\Shravtek\neutech_airfillter\anamolib\anomalib


In [4]:
pwd

'd:\\Shravtek\\neutech_airfillter\\anamolib\\anomalib'

In [5]:
# Import the datamodule
# Import the datamodule
from anomalib.data import Folder
from anomalib.data.utils import TestSplitMode

# Create the datamodule
datamodule = Folder(
    name="hazelnut_toy",
    root="datasets/hazelnut_toy",
    normal_dir="good",
    test_split_mode=TestSplitMode.SYNTHETIC,
    image_size=(512,512)
)

# Setup the datamodule
datamodule.setup()

In [6]:
i, train_data = next(enumerate(datamodule.train_dataloader()))
print(train_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, val_data = next(enumerate(datamodule.val_dataloader()))
print(val_data.keys())
# dict_keys(['image_path', 'label', 'image'])

i, test_data = next(enumerate(datamodule.test_dataloader()))
print(test_data.keys())
# dict_keys(['image_path', 'label', 'image'])

dict_keys(['image_path', 'label', 'image', 'mask'])
dict_keys(['image_path', 'label', 'image', 'mask'])
dict_keys(['image_path', 'label', 'image', 'mask'])


In [8]:
model = Padim()

In [ ]:
# # start training
# engine = Engine(task=TaskType.SEGMENTATION,accelerator="auto",
#     check_val_every_n_epoch=1,
#     max_epochs=2,
#     num_sanity_val_steps=0,
#     val_check_interval=1.0
#     )
# engine.fit(model=model, datamodule=datamodule)

In [9]:
# start training
engine = Engine(task=TaskType.SEGMENTATION
    )
engine.fit(model=model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PadimModel               | 2.8 M 
1 | _transform            | Compose                  | 0     
2 | normalization_metrics | MinMax                

Training: |          | 0/? [00:00<?, ?it/s]

c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\loops\optimization\automatic.py:129: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


In [12]:
test_results = engine.test(
    model=model,
    datamodule=datamodule,
    ckpt_path=engine.trainer.checkpoint_callback.best_model_path,
)

F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
F1Score class exists for backwards compatibility. It will be removed in v1.1. Please use BinaryF1Score from torchmetrics instead
Restoring states from the checkpoint path at D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Padim\hazelnut_toy\v5\weights\lightning\model.ckpt
Loaded model weights from the checkpoint at D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Padim\hazelnut_toy\v5\weights\lightning\model.ckpt
c:\Users\shan\.conda\envs\anamo\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        image_AUROC        │           0.75            │
│       image_F1Score       │            0.0            │
│        pixel_AUROC        │     0.757884681224823     │
│       pixel_F1Score       │    0.13023769855499268    │
└───────────────────────────┴───────────────────────────┘

In [10]:
test_results

[{'pixel_AUROC': 0.757884681224823,
  'pixel_F1Score': 0.13023769855499268,
  'image_AUROC': 0.75,
  'image_F1Score': 0.0}]

In [12]:
engine.export(
    model=model,
    export_type=ExportType.OPENVINO,
)

c:\Users\shan\.conda\envs\anamo\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
c:\Users\shan\.conda\envs\anamo\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(
c:\Users\shan\.conda\envs\anamo\lib\site-packages\torch\onnx\utils.py:1209: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ..\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C.

WindowsPath('D:/Shravtek/neutech_airfillter/anamolib/anomalib/results/Padim/hazelnut_toy/latest/weights/openvino/model.xml')

In [13]:
image_path = "D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/hazelnut_toy/crack/01.jpg"
image = read_image(path="D:/Shravtek/neutech_airfillter/anamolib/anomalib/datasets/hazelnut_toy/crack/01.jpg")
plt.imshow(image)

In [14]:
output_path = Path(engine.trainer.default_root_dir)
print(output_path)

D:\Shravtek\neutech_airfillter\anamolib\anomalib\results\Padim\hazelnut_toy\latest


In [15]:
openvino_model_path = output_path / "weights" / "openvino" / "model.bin"
metadata = output_path / "weights" / "openvino" / "metadata.json"
print(openvino_model_path.exists(), metadata.exists())

True True


In [16]:
inferencer = OpenVINOInferencer(
    path=openvino_model_path,  # Path to the OpenVINO IR model.
    metadata=metadata,  # Path to the metadata file.
    device="CPU",  # We would like to run it on an Intel CPU.
)

In [23]:
predictions = inferencer.predict(image=image_path)

In [24]:
print(predictions.pred_score, predictions.pred_label)

0.4972085171740765 LabelName.NORMAL


In [18]:
# Visualize the original image
print(predictions)

ImageResult(image=[[[228 214  79]
  [231 217  82]
  [231 217  82]
  ...
  [226 213  82]
  [225 213  79]
  [229 217  83]]

 [[228 214  79]
  [228 214  79]
  [229 215  80]
  ...
  [226 214  80]
  [227 215  81]
  [228 216  80]]

 [[230 217  79]
  [230 217  79]
  [229 216  78]
  ...
  [225 213  79]
  [226 214  78]
  [222 210  74]]

 ...

 [[233 224  83]
  [232 223  82]
  [232 223  82]
  ...
  [223 211  73]
  [223 211  73]
  [222 210  72]]

 [[233 224  83]
  [229 220  79]
  [235 226  85]
  ...
  [225 214  73]
  [223 212  71]
  [224 213  72]]

 [[230 221  80]
  [230 221  80]
  [231 222  81]
  ...
  [219 208  67]
  [221 210  69]
  [226 215  74]]], pred_score=1.0, pred_label=1, anomaly_map=[[0.2643897  0.26474822 0.26583838 ... 0.26394427 0.2615446  0.2607414 ]
 [0.2644479  0.2648059  0.2658955  ... 0.2640093  0.2616168  0.26081592]
 [0.26464215 0.26499903 0.2660864  ... 0.26421773 0.2618471  0.2610533 ]
 ...
 [0.27315807 0.2730011  0.27252084 ... 0.23491961 0.23412982 0.2338576 ]
 [0.27061862

In [19]:
# Visualize the segmentation mask.
plt.imshow(predictions.pred_mask)
